In [ ]:
import pandas
import numpy
import sys
import re
'''Initial ingestion of data and then counting the total number of instances in the whole data set'''
def data_process(input):
 with open(input) as file:
 files = file.readlines()
 no_of_instances, no_of_instances2, no_of_empty_lines, ii = 0, 0, 0, 0
 line_contents, line_contents2, sector_contents, list_of_line_contents = list(), list(), list(),
list()
 for line in files:
 no_of_instances += 1
 if len(line) <= 1:
 no_of_empty_lines += 1
 line_contents.append(line.split())
 for i in line_contents:
 if len(i) == 0:
 ii += 1
 line_contents.remove(i)
 sector_contents.append(list_of_line_contents)
 list_of_line_contents = []
 else:
 no_of_instances2 += 1
 line_contents2.append(i)
 list_of_line_contents.append(i)
 No_of_Instances_in_Original_File = len(sector_contents)
 return sector_contents, No_of_Instances_in_Original_File
'''Filtering out the unwanted items in an instance and count the initial count of each sense
type'''
def data_segment(list_of_features):
 for i in range(len(list_of_features)):
 if len(list_of_features[i]) == 5:
 list_of_features[i].remove(['<context>'])
 list_of_features[i].remove(['</context>'])
 list_of_features[i].remove(['</instance>'])
 elif len(list_of_features[i]) == 6:
 list_of_features[i].pop(0)
 list_of_features[i].remove(['</instance>'])
 list_of_features[i].remove(['<context>'])
 list_of_features[i].remove(['</context>'])
 Initial_label_Dict = dict()
 ''' Count of number of instances for each class in the original data set'''
 Prob_sense_dict = dict()
 for i in range(len(list_of_features)):
 item = list_of_features[i][0][2].split('=')[-1].split('%')[-1].strip('"/>')
 if item in Initial_label_Dict.keys():
 Initial_label_Dict[item] = Initial_label_Dict[item] + 1
 elif item not in Initial_label_Dict.keys():
 Initial_label_Dict[item] = 1
 for item in Initial_label_Dict.keys():
 Prob_sense_dict[item] = Initial_label_Dict[item] / sum(Initial_label_Dict.values())
 list_of_features_with_label1 = []
 list_of_features_with_label2 = []
 label_feature_dict = dict()
 for i in range(len(list_of_features)):
 item = list_of_features[i][0][2].split('=')[-1].split('%')[-1].strip('"/>')
 for j in Initial_label_Dict.keys():
 if item == j and (item == 'factory' or item == 'fish' or item == 'machine' or item ==
'physical' or item == 'hand' or item == 'container'):
 list_of_features_with_label1.append(list_of_features[i][1])
 label_feature_dict[item] = list_of_features_with_label1
 elif item == 'living' or item == 'music' or item == 'bird' or item == 'legal' or item == 'tree'
or item == 'vehicle':
 list_of_features_with_label2.append(list_of_features[i][1])
 label_feature_dict[item] = list_of_features_with_label2
 ''' Copy similar for other data sets'''
 # print(list_of_features)
 complete_feature_list = dict()
 complete_feature_list_for_label1 = dict()
 complete_feature_list_for_label2 = dict()
 ''' Dictionary of all words in the complete data set'''
 for i in range(len(list_of_features)):
 for j in range(len(list_of_features[i][1])):
 word = re.sub(r'[^-\w]', '', list_of_features[i][1][j])
 # print(word)
 if word in complete_feature_list.keys():
 complete_feature_list[word] = complete_feature_list[word] + 1
 elif word not in complete_feature_list.keys():
 complete_feature_list[word] = 1
 # print(len(complete_feature_list))
 ''' Dictionary of all the features for sense 1'''
 for i in range(len(list_of_features_with_label1)):
 for j in range(len(list_of_features_with_label1[i])):
 word = re.sub(r'[^-\w]', '', list_of_features_with_label1[i][j])
 if word not in complete_feature_list_for_label1.keys():
 complete_feature_list_for_label1[word] = 1
 elif word in complete_feature_list_for_label1.keys():
 complete_feature_list_for_label1[word] = complete_feature_list_for_label1[word] + 1
 # print(len(complete_feature_list_for_label1))
 ''' Dictionary of all the features for sense 2'''
 for i in range(len(list_of_features_with_label2)):
 for j in range(len(list_of_features_with_label2[i])):
 word = re.sub(r'[^-\w]', '', list_of_features_with_label2[i][j])
 if word not in complete_feature_list_for_label2.keys():
 complete_feature_list_for_label2[word] = 1
 elif word in complete_feature_list_for_label2.keys():
 complete_feature_list_for_label2[word] = complete_feature_list_for_label2[word] + 1
 # print(len(complete_feature_list_for_label2))
 return list_of_features, complete_feature_list, complete_feature_list_for_label1,
complete_feature_list_for_label2, Initial_label_Dict, Prob_sense_dict
def cross_fold_creation(list_of_features):
 extra = len(list_of_features) % 5
 fold1, fold2, fold3, fold4, fold5 = [], [], [], [], []
 new_length = 0
 if extra != 0:
 new_length = len(list_of_features) + (5 - extra)
 else:
 new_length = len(list_of_features)
 fold_lenght = int(new_length / 5)
 # print(fold_length)
 fold1 = list_of_features[0:fold_lenght]
 fold2 = list_of_features[fold_lenght: 2 * fold_lenght]
 fold3 = list_of_features[2 * fold_lenght: 3 * fold_lenght]
 fold4 = list_of_features[3 * fold_lenght: 4 * fold_lenght]
 fold5 = list_of_features[4 * fold_lenght:]
 # print(len(fold1), len(fold2), len(fold3), len(fold4), len(fold5))
 return fold1, fold2, fold3, fold4, fold5
def creating_train_test_segments(fold1, fold2, fold3, fold4, fold5):
 list_of_folds = [fold1, fold2, fold3, fold4, fold5]
 list_of_train_sets = []
 list_of_test_sets = []
 ''' Adding the list of test folds to a list of all test folds'''
 for i in range(5):
 list_of_test_sets.append(list_of_folds[i])
 for i in range(5):
 list_of_intermediate_train_sets = []
 # print('Round number', i)
 for j in range(5):
 if list_of_test_sets[i] == list_of_folds[j]:
 # print('Fold ignored:', i)
 pass
 else:
 # print('Fold added:', j)
 list_of_intermediate_train_sets.append(list_of_folds[j])
 # print('------------------')
 # print(len(list_of_intermediate_train_sets[3]))
 list_of_train_sets.append(list_of_intermediate_train_sets)
 return list_of_train_sets, list_of_test_sets, list_of_folds
def calculating_features_probabilities_for_each_set(list_of_train_sets, list_of_features, lenS1,
lenS2, Initial_Sense_Dict, Prob_Sense_Dict):
 ''' Extracting Target word between the <head> and </head> tags, also listing all the features
in all the train folds'''
 list_of_target_words_for_all_folds = list()
 list_of_features_for_all_folds = list()
 for i in range(len(list_of_train_sets)):
 list_of_target_words = list()
 list_of_features_for_each_fold = list()
 for j in range(len(list_of_train_sets[i])):
 for k in range(len(list_of_train_sets[i][j])):
 # print(list_of_train_sets[i][j][k][1])
 list_under_consideration = list_of_train_sets[i][j][k][1]
 for l in range(len(list_under_consideration)):
 if list_under_consideration[l].split('d>')[0] == '<hea':
 word = list_under_consideration[l].split('<head>')[-1].split('</head>')[0]
 list_of_target_words.append(word)
 else:
 list_of_features_for_each_fold.append(list_under_consideration[l])
 list_of_target_words_for_all_folds.append(list_of_target_words)
 list_of_features_for_all_folds.append(list_of_features_for_each_fold)
 # print(len(list_of_features_for_all_folds[4]))
 ''' Calculating V1, V2 for add one smoothing in each train set'''
 unique_sense1_word_count = []
 unique_sense2_word_count = []
 for i in range(len(list_of_train_sets)):
 Word_count = dict()
 Word_count2 = dict()
 for j in range(len(list_of_train_sets[i])):
 for k in range(len(list_of_train_sets[i][j])):
 sense = list_of_train_sets[i][j][k][0][-1].split('="')[-1].split('%')[-1].strip('"/>')
 if sense == 'factory' or sense == 'fish' or sense == 'machine' or sense == 'physical' or
sense == 'hand' or sense == 'container':
 for l in range(len(list_of_train_sets[i][j][k][1])):
 word = re.sub(r'[^-\w]', '', list_of_train_sets[i][j][k][1][l])
 if word not in Word_count.keys():
 Word_count[word] = 1
 elif word in Word_count.keys():
 Word_count[word] = Word_count[word] + 1
 elif sense == 'living' or sense == 'music' or sense == 'bird' or sense == 'legal' or sense ==
'tree' or sense == 'vehicle':
 for l in range(len((list_of_train_sets[i][j][k][1]))):
 word = re.sub(r'[^-\w]', '', list_of_train_sets[i][j][k][1][l])
 if word not in Word_count2.keys():
 Word_count2[word] = 1
 elif sense in Word_count2.keys():
 Word_count2[word] = Word_count2[word] + 1
 unique_sense1_word_count.append(len(Word_count))
 unique_sense2_word_count.append(len(Word_count2))
 # print('qwertyuiop', unique_sense1_word_count)
 '''Dictionary containing count of each word in each sense and the counting the probabilities'''
 List_of_all_feature_count_dictionaries = list()
 for i in range(len(list_of_features_for_all_folds)):
 Dictionary_of_counts = dict()
 for j in range(len(list_of_features_for_all_folds[i])):
 word = re.sub(r'[^-\w]', '', list_of_features_for_all_folds[i][j])
 if word in Dictionary_of_counts.keys():
 Dictionary_of_counts[word] = Dictionary_of_counts[word] + 1
 elif word not in Dictionary_of_counts.keys():
 Dictionary_of_counts[word] = 1
 List_of_all_feature_count_dictionaries.append(Dictionary_of_counts)
 # print('qwert',List_of_all_feature_count_dictionaries[0])
 ''' Create a list of features for different labels'''
 List_of_all_feature_count_for_sense1 = list()
 List_of_all_feature_count_for_sense2 = list()
 for i in range(len(list_of_train_sets)):
 List_of_fold_feature_count_for_sense1 = list()
 List_of_fold_feature_count_for_sense2 = list()
 for j in range(len(list_of_train_sets[i])):
 for k in range(len(list_of_train_sets[i][j])):
 sentence = list_of_train_sets[i][j][k][0]
 sense = sentence[-1].split('="')[-1].split('%')[-1].strip('"/>')
 for l in range(len(list_of_train_sets[i][j][k][1])):
 word = re.sub(r'[^\w-]', '', list_of_train_sets[i][j][k][1][l])
 if sense == 'living' or sense == 'music' or sense == 'bird' or sense == 'legal' or sense ==
'tree' or sense == 'vehicle':
 List_of_fold_feature_count_for_sense2.append(word)
 elif sense == 'factory' or sense == 'fish' or sense == 'machine' or sense == 'physical' or
sense == 'hand' or sense == 'container':
 List_of_fold_feature_count_for_sense1.append(word)
 List_of_all_feature_count_for_sense1.append(List_of_fold_feature_count_for_sense1)
 List_of_all_feature_count_for_sense2.append(List_of_fold_feature_count_for_sense2)
 ''' Creating a Dictionary counts of features for each fold and each label'''
 List_of_all_features_count_for_sense1 = list()
 List_of_all_features_count_for_sense2 = list()
 for i in range(len(List_of_all_feature_count_for_sense1)):
 Dict_of_counts_in_each_train_set_for_sense1 = dict()
 for j in range(len(List_of_all_feature_count_for_sense1[i])):
 word = re.sub(r'[^-\w]', '', List_of_all_feature_count_for_sense1[i][j])
 if word in Dict_of_counts_in_each_train_set_for_sense1.keys():
 Dict_of_counts_in_each_train_set_for_sense1[word] =
Dict_of_counts_in_each_train_set_for_sense1[word] + 1
 elif word not in Dict_of_counts_in_each_train_set_for_sense1.keys():
 Dict_of_counts_in_each_train_set_for_sense1[word] = 1

List_of_all_features_count_for_sense1.append(Dict_of_counts_in_each_train_set_for_sense1)
 # print(List_of_all_features_count_for_sense1[0])
 for i in range(len(List_of_all_feature_count_for_sense2)):
 Dict_of_counts_in_each_train_set_for_sense2 = dict()
 for j in range(len(List_of_all_feature_count_for_sense2[i])):
 word = re.sub(r'[^\w-]', '', List_of_all_feature_count_for_sense2[i][j])
 if word not in Dict_of_counts_in_each_train_set_for_sense2.keys():
 Dict_of_counts_in_each_train_set_for_sense2[word] = 1
 elif word in Dict_of_counts_in_each_train_set_for_sense2.keys():
 Dict_of_counts_in_each_train_set_for_sense2[word] =
Dict_of_counts_in_each_train_set_for_sense2[word] + 1

List_of_all_features_count_for_sense2.append(Dict_of_counts_in_each_train_set_for_sense2)
 # print(List_of_all_features_count_for_sense2[0])
 '''Calculating the number of sense instances in each train set'''
 Sense1_Instances = []
 Sense2_Instances = []
 for i in range(len(list_of_train_sets)):
 sense_count_dict = dict()
 for j in range(len(list_of_train_sets[i])):
 for k in range(len(list_of_train_sets[i][j])):
 list_to_be_parsed = list_of_train_sets[i][j][k][0]
 sense = list_to_be_parsed[-1].split('="')[-1].split('%')[-1].strip('"/>')
 if sense not in sense_count_dict.keys():
 sense_count_dict[sense] = 1
 elif sense in sense_count_dict.keys():
 sense_count_dict[sense] = sense_count_dict[sense] + 1
 Sense1_Instances.append(sense_count_dict['factory'])
 Sense2_Instances.append(sense_count_dict['living'])
 # Sense1_Instances.append(sense_count_dict['fish'])
 # Sense2_Instances.append(sense_count_dict['music'])
 # Sense1_Instances.append(sense_count_dict['machine'])
 # Sense2_Instances.append(sense_count_dict['bird'])
 # Sense1_Instances.append(sense_count_dict['physical'])
 # Sense2_Instances.append(sense_count_dict['legal'])
 # Sense1_Instances.append(sense_count_dict['hand'])
 # Sense2_Instances.append(sense_count_dict['tree'])
 # Sense1_Instances.append(sense_count_dict['container'])
 # Sense2_Instances.append(sense_count_dict['vehicle'])
 # print('Sense :', Sense1_Instances)
 ''' Dictionary of all the values for each feature and each sense'''
 Word_Sense1_Prob_Dict = dict()
 Word_Sense2_Prob_Dict = dict()
 for i in range(len(List_of_all_features_count_for_sense1)):
 for key in List_of_all_features_count_for_sense1[i].keys():
 Word_Sense1_Prob_Dict[key] = List_of_all_features_count_for_sense1[i][key] /
Sense1_Instances[i]
 for i in range(len(List_of_all_features_count_for_sense2)):
 for key in List_of_all_features_count_for_sense2[i].keys():
 Word_Sense2_Prob_Dict[key] = List_of_all_features_count_for_sense2[i][key] /
Sense2_Instances[i]
 '''Modifying the Dictionaries created above for words not present with certain label'''
 for i in range(len(list_of_train_sets)):
 ss = Sense1_Instances[i]
 ss2 = Sense2_Instances[i]
 for word in list_of_features.keys():
 if word not in Word_Sense2_Prob_Dict.keys():
 Word_Sense2_Prob_Dict[word] = (list_of_features[word] + 1) / (ss2 +
unique_sense2_word_count[i])
 if word not in Word_Sense1_Prob_Dict.keys():
 Word_Sense1_Prob_Dict[word] = (list_of_features[word] + 1) / (ss +
unique_sense1_word_count[i])
 return Word_Sense1_Prob_Dict, Word_Sense2_Prob_Dict
def Test_Prob_Computation(Word_Sense1_Prob_Dict, Word_Sense2_Prob_Dict,
list_of_test_sets, Prob_Sense_Dict):
 Instance = []
 Original_Sense = []
 for i in range(len(list_of_test_sets)):
 list_of_Instances_for_each_fold = []
 list_of_Original_Sense_for_each_fold = []
 for j in range(len(list_of_test_sets[i])):
 item = list_of_test_sets[i][j][0]
 list_of_Instances_for_each_fold.append(item[1].split('="')[-1].strip('"'))
 list_of_Original_Sense_for_each_fold.append(item[2].split('="')[-1].strip('"/>'))
 Instance.append(list_of_Instances_for_each_fold)
 Original_Sense.append(list_of_Original_Sense_for_each_fold)
 Calculated_Sense = dict()
 llist = []
 for i in range(len(list_of_test_sets)):
 list_of_actual_answers = []
 for j in range(len(list_of_test_sets[i])):
 overall_prob_for_sense1 = 0
 overall_prob_for_sense2 = 0
 Instance = list_of_test_sets[i][j][0][1].split('="')[-1].strip('"')
 Actual_Sense = list_of_test_sets[i][j][0][2].split('="')[-1].split('%')[-1].strip('"/>')
 Dict = dict()
 for k in range(len(list_of_test_sets[i][j][1])):
 word = re.sub(r'[^-\w]', '', list_of_test_sets[i][j][1][k])
 # print(word, overall_prob_for_sense1, i, j, k)
 overall_prob_for_sense1 += numpy.log2(Word_Sense1_Prob_Dict[word])
 overall_prob_for_sense1 = overall_prob_for_sense1 * Prob_sense_Dict['factory']
 # overall_prob_for_sense1 = overall_prob_for_sense1 * Prob_sense_Dict['fish']
 # overall_prob_for_sense1 = overall_prob_for_sense1 * Prob_sense_Dict['machine']
 # overall_prob_for_sense1 = overall_prob_for_sense1 * Prob_sense_Dict['physical']
 # overall_prob_for_sense1 = overall_prob_for_sense1 * Prob_sense_Dict['hand']
 # overall_prob_for_sense1 = overall_prob_for_sense1 * Prob_sense_Dict['container']
 Dict['factory'] = overall_prob_for_sense1
 # Dict['fish'] = overall_prob_for_sense1
 # Dict['machine'] = overall_prob_for_sense1
 # Dict['physical'] = overall_prob_for_sense1
 # Dict['hand'] = overall_prob_for_sense1
 # Dict['container'] = overall_prob_for_sense1
 # print(overall_prob_for_sense1)
 for k in range(len(list_of_test_sets[i][j][1])):
 word = re.sub(r'[^-\w]', '', list_of_test_sets[i][j][1][k])
 overall_prob_for_sense2 += numpy.log2(Word_Sense2_Prob_Dict[word])
 overall_prob_for_sense2 = overall_prob_for_sense2 * Prob_sense_Dict['living']
 # overall_prob_for_sense2 = overall_prob_for_sense2 * Prob_sense_Dict['music']
 # overall_prob_for_sense2 = overall_prob_for_sense1 * Prob_sense_Dict['bird']
 # overall_prob_for_sense2 = overall_prob_for_sense2 * Prob_sense_Dict['legal']
 # overall_prob_for_sense2 = overall_prob_for_sense2 * Prob_sense_Dict['tree']
 # overall_prob_for_sense2 = overall_prob_for_sense2 * Prob_sense_Dict['vehicle']
 Dict['living'] = overall_prob_for_sense2
 # Dict['music'] = overall_prob_for_sense2
 # Dict['bird'] = overall_prob_for_sense2
 # Dict['tree'] = overall_prob_for_sense2
 # Dict['legal'] = overall_prob_for_sense2
 # Dict['vehicle'] = overall_prob_for_sense2
 # print(Dict, max(Dict, key=Dict.get))
 Calculated_Sense[Instance] = max(Dict, key=Dict.get)
 list_of_actual_answers.append(max(Dict, key=Dict.get))
 llist.append(list_of_actual_answers)
 correct_response = 0
 total_count = 0
 list_to_be_used_in_dataframe = []
 for i in range(len(llist)):
 list_to_be_used_in_dataframe += [["Fold {}".format(i+1)] + [' ']]
 correct_response1 = 0
 total_count1 = 0
 for j in range(len(llist[i])):
 if llist[i][j] == Original_Sense[i][j].split('%')[-1]:
 correct_response += 1
 correct_response1 += 1
 total_count += 1
 total_count1 += 1
 list_to_be_used_in_dataframe += [[list_of_test_sets[i][j][0][1].split('="')[-1].strip('"')] +
['plant%'+Calculated_Sense[list_of_test_sets[i][j][0][1].split('="')[-1].strip('"')]]]
 else:
 list_to_be_used_in_dataframe += [[list_of_test_sets[i][j][0][1].split('="')[-1].strip('"')] +
['plant%'+Calculated_Sense[list_of_test_sets[i][j][0][1].split('="')[-1].strip('"')]]]
 total_count += 1
 total_count1 += 1
 print('Accuracy for Fold{} is {}%'.format(i, round(100*(correct_response1/total_count1),
2)))
 print('Overall Accuracy is {}%'.format(round(100*(correct_response / total_count), 2)))
 
if __name__ == '__main__':
 input = sys.argv[1]
 a, b = data_process(input)
 c, lenS1_S2, lenS1, lenS2, Initial_label_dict, Prob_sense_Dict = data_segment(a)
 f1, f2, f3, f4, f5 = cross_fold_creation(c)
 train_features, test, fold_list = creating_train_test_segments(f1, f2, f3, f4, f5)
 Word_Sense_Prob1, Word_Sense_Prob2 =
calculating_features_probabilities_for_each_set(train_features, lenS1_S2, lenS1, lenS2,
Initial_label_dict, Prob_sense_Dict)
 Test_Prob_Computation(Word_Sense_Prob1, Word_Sense_Prob2, test, Prob_sense_Dict)